# Rossmann Store Sales

- Bases de dados: https://www.kaggle.com/c/rossmann-store-sales/data


O uso de Machine Learning é importante para aumentar a competitividade das empresas ao prever vendas futuras. Essas bases de dados contém históricos de vendas de 1.115 lojas Rossmann.

**O objetivo deste projeto é prever vendas diárias futuras.**

Modelos de previsão de vendas baseados em dados passados devem considerar o que chamamos de efeitos sazonais, que são efeitos como demanda, feriados, concorrência e promoções.

As base de dados contém histórico de transações e dados das lojas:

- **id**- identificador da transação (loja+data)
- **Store** - identificador único para cada loja
- **Sales** - o volume de negócios/vendas para um determinado dia (objetivo)
- **Customers** - o número de clientes em um determinado dia
- **Open** - um indicador para saber se a loja estava aberta: 0 = fechado, 1 = aberto
- **StateHoliday** - indica um feriado estadual. Normalmente todas as lojas, com poucas exceções, fecham nos feriados estaduais. Observe que todas as escolas fecham nos feriados e fins de semana. 
 - a = feriado, b = feriado da Páscoa, c = Natal, 0 = nenhum

- **SchoolHoliday** - feriado escolar
- **StoreType** - diferencia entre 4 modelos de loja diferentes: a, b, c, d
- **Assortment** - descreve um nível de sortimento: 
  - a = básico, b = extra, c = estendido
- **CompetitionDistance** - distância em metros até a loja concorrente mais próxima
- **CompetitionOpenSince[Month/Year]** - fornece o ano e o mês aproximados da hora em que o concorrente mais próximo foi aberto
- **Promo** - indica se uma loja está realizando uma promoção naquele dia
- **Promo2** - Promo2 é uma promoção contínua e consecutiva para algumas lojas: 
  - 0 = loja não está participando, 1 = loja está participando
- **Promo2Since[Year/Week]** - descreve o ano e a semana em que a loja começou a participar do Promo2
- **PromoInterval** - descreve os intervalos consecutivos em que o Promo2 é iniciado, nomeando os meses em que a promoção é reiniciada. Por exemplo. "Fevereiro, maio, agosto, novembro" significa que cada rodada começa em fevereiro, maio, agosto, novembro de qualquer ano para essa loja



In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
import warnings
warnings.filterwarnings('ignore')

In [2]:
df_stores = pd.read_csv('stores_cleaning.csv')

In [3]:
df_stores.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,MonthMap,IsPromo
0,1,5,2015-07-31,5263,555,1,1,0,1,c,a,1270.0,9.0,2008.0,0,31.0,2015.0,0,Jul,0
1,2,5,2015-07-31,6064,625,1,1,0,1,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct",Jul,1
2,3,5,2015-07-31,8314,821,1,1,0,1,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct",Jul,1
3,4,5,2015-07-31,13995,1498,1,1,0,1,c,c,620.0,9.0,2009.0,0,31.0,2015.0,0,Jul,0
4,5,5,2015-07-31,4822,559,1,1,0,1,a,a,29910.0,4.0,2015.0,0,31.0,2015.0,0,Jul,0


Mudança de tipos

In [4]:
df_stores['Date'] = pd.to_datetime(df_stores['Date'])
df_stores['CompetitionOpenSinceMonth'] = df_stores['CompetitionOpenSinceMonth'].astype( int )
df_stores['CompetitionOpenSinceYear'] = df_stores['CompetitionOpenSinceYear'].astype(int)
# promo2
df_stores['Promo2SinceWeek'] = df_stores['Promo2SinceWeek'].astype( int )
df_stores['Promo2SinceYear'] = df_stores['Promo2SinceYear'].astype( int )

Derivando atributos a partir da coluna `Date`

In [5]:
# year
df_stores['Year'] = df_stores['Date'].dt.year
# month
df_stores['Month'] = df_stores['Date'].dt.month
# day
df_stores['Day'] = df_stores['Date'].dt.day
# week of year
df_stores['WeekOfYear'] = df_stores['Date'].dt.weekofyear
# year week
df_stores['YearWeek'] = df_stores['Date'].dt.strftime( '%Y-%W' )

In [6]:
# competition since
df_stores['CompetitionSince'] = df_stores.apply( lambda x: datetime.datetime(year=x['CompetitionOpenSinceYear'],
month=x['CompetitionOpenSinceMonth'],day=1 ), axis=1 )

In [9]:
df_stores[['CompetitionOpenSinceYear','CompetitionOpenSinceMonth','CompetitionSince']].head(3)

,CompetitionOpenSinceYear,CompetitionOpenSinceMonth,CompetitionSince
0,2008,9,2008-09-01
1,2007,11,2007-11-01
2,2006,12,2006-12-01


In [10]:
df_stores['CompetitionTimeMonth'] = ( ( df_stores['Date'] - df_stores['CompetitionSince'] )/30).apply(
     lambda x: x.days ).astype( int )

In [12]:
df_stores[['Date','CompetitionSince','CompetitionTimeMonth']].head(3)

,Date,CompetitionSince,CompetitionTimeMonth
0,2015-07-31,2008-09-01,84
1,2015-07-31,2007-11-01,94
2,2015-07-31,2006-12-01,105


In [13]:
# promo since
df_stores['PromoSince'] = df_stores['Promo2SinceYear'].astype( str ) + '-' + df_stores['Promo2SinceWeek'].astype( str )
df_stores['PromoSince'] = df_stores['PromoSince'].apply( lambda x: datetime.datetime.strptime(
     x + '-1', '%Y-%W-%w' ) - datetime.timedelta( days=7 ) )
df_stores['PromoTimeWeek'] = ( ( df_stores['Date'] - df_stores['PromoSince'] )/7 ).apply(
    lambda x: x.days ).astype( int )
# assortment
df_stores['Assortment'] = df_stores['Assortment'].apply( lambda x: 'basic' if x == 'a' else 
                                                        'extra' if x == 'b' else 'extended' )
# state holiday
df_stores['StateHoliday'] = df_stores['StateHoliday'].apply( 
    lambda x: 'PublicHoliday' if x == 'a' else 'EasterHoliday' if x == 'b' else 'Christmas' if x == 'c'
else 'RegularDay' )

In [14]:
df_stores.head().T

,0,1,2,3,4
Store,1,2,3,4,5
DayOfWeek,5,5,5,5,5
Date,2015-07-31 00:00:00,2015-07-31 00:00:00,2015-07-31 00:00:00,2015-07-31 00:00:00,2015-07-31 00:00:00
Sales,5263,6064,8314,13995,4822
Customers,555,625,821,1498,559
Open,1,1,1,1,1
Promo,1,1,1,1,1
StateHoliday,RegularDay,RegularDay,RegularDay,RegularDay,RegularDay
SchoolHoliday,1,1,1,1,1
StoreType,c,a,a,c,a


### Filtragem das variáveis

#### Filtrando linhas

- A coluna `Open` representa se a loja estava aberta ou não, então as linhas em que Open for 0 não nos interessa, pois não há vendas
- Quando a loja está fechada então as vendas são 9, logo, não precisaremos das linhas em que o valor de `Sales` é 0

In [15]:
df_stores = df_stores[(df_stores['Open'] != 0) & (df_stores['Sales'] > 0)]

#### Filtrando colunas

- A coluna `Customers` representa o número de clientes em um dia. Essa coluna não fará sentido para a previsão das vendas, pois se formos prever as vendas para os próximos 6 meses, então também teríamos que ter o número de clientes para os próximos 6 meses, e isso não será possível, pois teríamos que também prever esse número de clientes
- Uma vez que selecionamos todos as linhas em que a coluna `Open`` era diferente de 0, então agora essa coluna não faz mais sentido, pois guarda apenas valores 1
- As colunas `PromoInterval` e `MonthMap` também não são mais necessárias, pois já derivamos outras colunas delas

In [16]:
cols_drop = ['Customers', 'Open', 'PromoInterval', 'MonthMap']
df_stores = df_stores.drop( cols_drop, axis=1 )

In [ ]:
df_stores.to_csv('final_data.csv',index=False)